In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from utils import calculate_qscore

# Load data

In [2]:
dataset_file = "data/labels/processed/global_mapped_cleaned.csv"  # full dataset
metadata_file = "data/svi/metadata.csv"

In [3]:
raw_df = pd.read_csv(dataset_file, index_col=0)
metadata_df = pd.read_csv(metadata_file, index_col=0)
metadata_df

,uuid,city,Relabelled Name,Image number
0,dc6eee81-9513-4edb-813b-05b83b5127be,Santiago,Image_1,1
1,50811b7a-5b0f-4581-92fa-017e6ff26ba0,Santiago,Image_2,2
2,35c8af9e-5e36-4810-9824-59b9b273fbcf,Santiago,Image_3,3
3,3a439403-305b-4be7-ba43-f8ff8e75c31c,Santiago,Image_4,4
4,3cc24150-c590-4d83-883c-44855010e338,Santiago,Image_5,5
...,...,...,...,...
395,416a4869-428a-475a-92bd-19f2611b2925,Amsterdam,Image_396,396
396,dad8c5a5-80cd-4d1e-a3cc-b429cdbb90c2,Amsterdam,Image_397,397
397,b7abb7f7-7031-4fc9-8f16-7e591dcdfbce,Amsterdam,Image_398,398
398,41a8b6ce-02df-47e8-b7d1-10cce55b6575,Amsterdam,Image_399,399


# Compute Q-score

Come up with a dataframe that have the scores for images based on the location of the SVI and location of participants

In [PP2](https://arxiv.org/abs/1608.01769), page 6, the max comparison across all indicators are:
- min 2.40
- avg 4.05, however on page 7 they state 3.35
- max 7.67 

In [4]:
to_from_df = pd.DataFrame()  # placeholder to keep track of data
min_filter = 4
countries = raw_df["Country"].unique()

for c in countries:
    country_df = raw_df[raw_df["Country"] == c]
    country_df_scores = calculate_qscore(country_df)
    country_df_scores_merged = pd.merge(
        country_df_scores,
        metadata_df,
        left_on="Image",
        right_on="Image number",
        how="left",
    )
    country_df_scores_merged["participants_from"] = c
    country_df_scores_merged["SVI_from"] = country_df_scores_merged["city"].map(
        {
            "Singapore": "Singapore",
            "San Francisco": "USA",
            "Amsterdam": "Netherlands",
            "Santiago": "Chile",
            "Abuja": "Nigeria",
        }
    )
    to_from_df = pd.concat([to_from_df, country_df_scores_merged])

to_from_df['Score'] = round(to_from_df['Score'], 2)
to_from_df.to_csv('data/labels/processed/to_from_scores.csv')

In [5]:
print(to_from_df['Num_comparisons'].min())
print(to_from_df['Num_comparisons'].max())

0
17
